# Announcements


# Boolean Logic and Assertions

<!-- <img src="img/true_or_false.gif"  width=600px /> -->
<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/true_or_false.gif"  width=600px />

## PHYS 2600: Scientific Computing

## Lecture 8

# FAQ: Vectorization

Vectorizing functions is useful when we want to define a function to accept and return single values, but later call it with an array of inputs and have it return an array.


Let's look at an example: the `dist_small_angle` function from homework 4. The total distance traveled by the pendulum bob after time $t$ is the integral
$$
D(t) = \int ds = \int L |d\theta| = \int dt \left(L \left| \frac{d\theta}{dt} \right|\right)= \theta_0 \sqrt{gL} \int_0^t dt' \left|\sin \left(\sqrt{\frac{g}{L}} t' \right) \right|.
$$



In [27]:
import matplotlib.pyplot as plt
import numpy as np

g = 9.8  # m/s^2
L = 2.0  # m
theta_0 = 10 * np.pi / 180  # 10 degrees


def dist_small_angle(t, N=50):
    t_int = np.linspace(0, t, N)
    I = np.trapezoid(np.abs(np.sin(np.sqrt(g / L) * t_int)), x=t_int)
    return theta_0 * np.sqrt(g * L) * I

If we assign a single value of `t` and calculate the distance, it works:

In [28]:
t = 2
print(dist_small_angle(t))

0.9488454248731222


But suppose we want to make a plot of `dist_small_angle` as a function of `t` (as you'll do in the homework). We'd like to call the function with an array, and that doesn't work correctly.

In [29]:
t_array = np.linspace(1, 5, num=5)  # array of t with 5 points from 1 to 5
print(t_array)

d_array = dist_small_angle(t_array)
print(d_array)
print(d_array.size)

[1. 2. 3. 4. 5.]
[0.         0.00850937 0.03357066 0.07380856 0.12701326 0.1902593
 0.26006277 0.33256827 0.40375542 0.46965377 0.52655428 0.57120608
 0.60098773 0.61404401 0.61394696 0.64464467 0.6622771  0.665065
 0.71455331 0.79353878 0.86049967 0.91139072 0.94253628 0.9508358
 1.01782181 1.0828912  1.12568911 1.14448342 1.15673097 1.22855663
 1.27746209 1.29953039 1.29152716 1.25109883 1.17694463 1.13743601
 1.30416143 1.44823161 1.56507177 1.6510403  1.70357079 1.72126534
 1.74500862 1.84514311 1.91039328 1.9357569  1.91735457 1.98082025
 2.07431295 2.12373944]
50


Our function isn't behaving the way we want it to! Instead of returning 5 values - the distance for each value in `t_array` - the function is returning 50 values. This is determined by the N=50 we set for the discretization, but I don't know what these values even are.

The problem is that our function isn't defined to accept an __array__ of `t`. This is a problem that vectorization can fix!

Let's see how things change when we vectorize the function.

In [30]:
d_func = np.vectorize(dist_small_angle)  # makes a vectorized function!

d_array = d_func(t_array)
print(d_array.size)

print(t_array)
print(d_array)

5
[1. 2. 3. 4. 5.]
[0.55821402 0.94884542 1.41860325 2.03829592 2.46590507]


Now things look correct! We can call `d_func` with an array `t_array`, and the function returns an array of distances, one for each value of t.

Note that np.vectorize doesn't __evaluate__ the function. It __creates__ a new function. `d_func` is not a variable!

You could also define your own vectorized wrapper function using a `for` loop.  
However, a more concise and often faster alternative is to use **list comprehension**:

In [31]:
def d_func(t_array, N=50):
    return np.array([dist_small_angle(t, N=N) for t in t_array])


d_array = d_func(t_array)
print(d_array.size)

print(t_array)
print(d_array)

5
[1. 2. 3. 4. 5.]
[0.55821402 0.94884542 1.41860325 2.03829592 2.46590507]


List comprehensions are generally faster than traditional `for` loops in Python due to internal optimizations.  

More importantly, they often improve **readability** by expressing the same logic in fewer lines of code.


## Introducing the Boolean type

__Boolean__ is the simplest form of data a computer can store: it has two possible values, `True` or `False`.  

(What do you expect is required for data storage of a Boolean variable?)

In Python, the name for the Boolean type is `bool`:

In [32]:
print(type(True), type(False))
print(bool(0), bool(1))

<class 'bool'> <class 'bool'>
False True


As you can see from the highlighting, the values `True` and `False` are built-in to Python as keywords.

Booleans are extremely important in programming!  They are used for __testing__ - you've seen this already in the "testing cells" on assignments.  If our program can test for truth, it can __branch__ based on the outcome of the test.

Going back to our very first lecture, we thought about the similarity between computer programs (which encode algorithms) and cooking with a recipe.  Here's a diagram of some instructions for making a meringue:

<!-- <img src="img/lec1-meringue.png" width=600px /> -->
<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/lec1-meringue.png" width=600px />

A key feature of more complex recipes (like this meringue cookie one) is __flow control__, which we see in two varieties above: __branching__ and __looping__.  Both require a conditional test!

## Booleans and comparison

Before we get to branching code, let's understand the Boolean tests themselves.  Python has a set of built-in __comparison operators__, which represent some True/False statement:

In [33]:
print(3 == 4)  # Equality
print(3 != 4)  # Inequality
print(4 < 4)  # Less than
print(4 <= 4)  # Less than or equal to
print(4 > 4)  # Greater than
print(4 >= 4)  # Greater than or equal to

False
True
False
True
False
True


These should be mostly familiar, but you'll probably have to get used to always using __double-equals__ `==` for testing equality!  (We already used `=` for assignment, of course.)

Since `True` and `False` are just data, we can save the outcome of a Boolean test to a variable for later use:

In [34]:
x = 3
x_is_zero = x == 0  # No () needed; = is always last!

print(x_is_zero)

False


Or we can return them from a function - useful if we want to run a more complicated test.

In [35]:
def is_zero(x):
    return x == 0


is_zero(3)

False

It is __very important to remember__ that like other Python features we've seen, comparisons are done _using the values_ at the moment we invoke them.

In other words, changing the variable _after_ a comparison won't change the comparison:

In [36]:
x = -3
x_is_positive = x > 0

print(x_is_positive)
x = 3  # changing x does not change the `x_is_positive` variable!
print(x_is_positive)  # still False
x_is_positive = x > 0
print(x_is_positive)

False
False
True


## Logical operators

What about more complicated tests, like `0<x<1`?  This is just two combined tests, `0<x` and `x<1`, using the "__and__" operation.

<!-- <a href="https://en.wikipedia.org/wiki/Boolean_algebra" target="_blank"><img src="img/venn_and_or_not.png" /></a> -->
<a href="https://en.wikipedia.org/wiki/Boolean_algebra" target="_blank"><img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/venn_and_or_not.png" /></a>


From left to right, the Venn diagrams above depict `and`, `or`, and `not` (written using standard logic notation.)  If you need a refresher on logic, the image links to the Wikipedia page it came from.  The corresponding Python operators are all built-in keywords that go by the same names:

In [37]:
print(True and False)  ## True if *both* statements are True
print(True or False)  ## True if *either* statement is True
print(not False)  ## True if the statement is False

False
True
True


To check the mathematical statement `0 < x < 1`, we would use `and`, since we want `x` to satisfy both limits:

In [38]:
x = 0.4
print(0 < x and x < 1)

True


Actually, Python has a _useful shorthand_ for chaining inequalities: we can simply write `0 < x < 1`, and it will be treated the same as the `and` version we wrote above!

In [39]:
print(0 < x < 1)

True


We have a lot of extra operators with Boolean operations!  We have to revise our [order of operations](https://www.programiz.com/python-programming/precedence-associativity) (a.k.a. operator precedence).  From first to last evaluated:

* Parentheses 
* Other math operations in usual order: `**`, `*` `/`, `+` `-`
* Comparisons (`==`, `<`, `>=`, ...)
* Logical operations in order: `not`, `and`, `or`
* Assignment (`=`)



In [40]:
y = 0.7
(y**2 < 1 - 0.5) and not (y - 1 > 0)  ## (0.49 < 0.5) and not (-0.3 > 0)

True

Automatic ordering makes for clean code, but when in doubt, just use parentheses!  (My example above is a nightmare for readability...)

## Assertions 

The simplest flow control keyword in Python is `assert`.  Assertion is for statements that __must be true__ - either for testing, or for verifying some condition you expect.

If the argument of `assert` evaluates to `True`, it does nothing; but if it's `False`, then an error message is raised and the program stops immediately!

In [41]:
assert 5 < 6  # no effect
assert 6 < 5  # error!
assert 5 == 5  # not evaluated

AssertionError: 

If we're writing a longer program and we want to stop it partway through while we're debugging, a simple way to do this is to simply write `assert False`, to halt execution no matter what.

`assert` is a __stand-alone expression__: it has to live on its own line (or you'll get a syntax error.)  As long as it's standing alone, we can use `assert` anywhere, even in a function.  Just remember, a failed `assert` stops the whole program!

In [42]:
def ensure_x_is_positive(x):
    assert x > 0
    print("Yes, x is positive!")


ensure_x_is_positive(3)
ensure_x_is_positive(-4)
print("This print won't happen.")

Yes, x is positive!


AssertionError: 

## Booleans as numbers

As mentioned above, although `bool` is a distinct type, in terms of storage a `bool` is just a single bit, `0` or `1`.  In fact, if we typecast those numbers we can see how they are interpreted:

In [43]:
print(f"bool(0): {bool(0)}")
print(f"bool(1): {bool(1)}")
print(f"bool(516): {bool(516)}")

bool(0): False
bool(1): True
bool(516): True


If we try to use anything which is _not_ `bool` for flow control, Python will automatically typecast (this feature is sometimes called "__truthiness__".)  Beware of unexpected behavior due to this!

In [44]:
assert 1
assert 0

AssertionError: 

## Tutorial 08

Time for some hands-on practice!  Connect to the server and load `tut08`.
